In [2]:
import re  # Just for checking if a string is an integer
import functools  # for product of list

test = "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2"

# Language definition of line:
# LINE =
#   INT |
#   (LINE) | 
#   LINE */+ LINE */+ ... */+ LINE

# This isn't  exactly correct... e.g. ((((3)) + (4))) will never appear but it does fit this language
# But that just means our language is broader which is fine


def matched_parentheses(line):
    # Maybe this could be done in regex?
    # We want to match (xxx) but not (xx) + (xx). So just looking at first/last char doesn't work
    if line[0] != '(' or line[-1] != ')':
        return False
    parentheses_level = 1
    for ix, c in enumerate(line[1:]):
        if c == '(':
            parentheses_level += 1
        elif c == ')':
            parentheses_level -= 1
        if parentheses_level == 0:
            return ix == len(line) -2  # double off-by-one :)

def split_lines(line):
    # Split LINE */+ LINE */+ LINE into [LINE, +/*, LINE, +/*, LINE]
    # simple splitting by */+ doesn't work because LINE could also contain */+
    # Could also maybe be done with regex but oh well
    result = []
    par_lvl = 0
    acc = ''
    for c in line:
        if c == '(':
            par_lvl += 1
            acc += c
        elif c == ')':
            par_lvl -= 1
            acc += c
        elif c == ' ' and par_lvl == 0:
            result.append(acc)
            acc = ''
        else:
            acc += c
    result.append(acc)
    return result

def apply_op(left, op, right):
    if op == '+':
        return left + right
    elif op == '*':
        return left * right
    raise ValueError("Op is {}, it should be + or *".format(op))

def parse(line):
    if re.match('^\d+$', line):
        return int(line)
    elif matched_parentheses(line):
        return parse(line[1:-1])
    else:
        split = split_lines(line)
        acc = parse(split[0])
        # convert list like ['3', '+', '4', '*', '2'] to 14
        for ix in range(len(split) // 2):
            acc = apply_op(acc, split[(2*ix)+1], parse(split[(2*ix)+2]))
        return acc


parse(test)  # Should be 13632

13632

In [3]:
with open('data/advent18.txt', 'r') as f:
    data = f.read().splitlines()

sum(map(parse, data))

800602729153

In [4]:
# Part 2:

def parse_2(line):
    if re.match('^\d+$', line):
        return int(line)
    elif matched_parentheses(line):
        return parse_2(line[1:-1])
    else:
        split = split_lines(line)
        # We are going to see if there are any '+' at the top level.
        # If so, wrap it in () so it gets precedence
        # Special case: len(split) == 3, so we don't infinitely keep wrapping the same thing
        if len(split) == 3:
            return apply_op(parse_2(split[0]), split[1], parse_2(split[2]))
        elif all(op == '*' for op in split[1:-1:2]):
            lines = [parse_2(l) for l in split[0::2]]
            return functools.reduce(lambda x, y: x*y, lines, 1)
        else:
            plus = split.index('+')  # only returns the first occurence which is what we want
            combined = "({} + {})".format(split[plus-1], split[plus+1])
            # now we have combined it, just recurse lol
            combined = ' '.join(split[:plus-1] + [combined] + split[plus+2:])
            return parse_2(combined)

parse_2(test)  # Should be 23340

23340

In [5]:
sum(map(parse_2, data))

92173009047076